In [1]:
import pandas as pd
import pickle
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import pylab as pl 
import sklearn.preprocessing
import seaborn as sns
from scipy import stats
import csv
#import tensorflow as tf
#from tensorflow import keras


pickle_in = open("flean", "rb")
fullclean = pickle.load(pickle_in)
fullclean = fullclean.loc[  fullclean['IND_MonthofDefault'] == 0]

FileNotFoundError: [Errno 2] No such file or directory: 'flean'

In [2]:
pd.set_option('display.max_columns', 125)
pd.set_option('display.max_rows', 501)
fullclean

,LoanNumber,PayDate,InterestRate,LoantoValue,Original_Amortization_Term,PMIPercent,Original_FICO_Min,TotalDebtratio,IncomePercentage,TotalOtherDebt,IND_TPO,IND_OBS2,PRODUCT,TOTALMonthlyIncome,OriginalPropertyValue,TotalLoantoValue_cleaned,IND_HBL,CurrentFlag,DLQ30Flag,DLQ60Flag,DLQ90OrPlusFlag,REOForeclosureFlag,DLQDays,Seasoning,RollingCurrentFlag,RollingDLQ30Flag,RollingDLQ60Flag,RollingDLQ90OrPlusFlag,RollingREOForeclosureFlag,ServicingModelName,OrigLoanAmount,BegMonthBal,RefiIncentive,SatO,RollingBurnout,RollingDLQDays,IND_MonthofDefault,IND_REOFCL_MOD,IND_180D_MOD,IND_LGD_MOD,IND_HLC_MOD,PrepayUPB,SMM,SMM_Flag,IND_prepay,FreddieMac_Pts_15Yr,FreddieMac_Pts_30Yr,FreddieMac_Rate_15Yr,FreddieMac_Rate_30Yr,IND_PMI,StateUnemploymentRate,ZillowHousingPrice,ZillowHousingPrice_MoverM_Change,ZillowHousingPrice_SinceOrigination_Change,FundingDate,SubProgramCode,LoanPurposeDescr,IND_FULL,FundingRefiPercentage,PaydateABV,PortfolioprepayPercentage,PercMortgageRemaining,XTRAIND,EastIND,SouthIND,MidwestIND,WestIND,PurchaseIND,CashoutRefIND,RateRefIND,NonSingleFamilyIND,NonprimaryIND
0,1568633,2015-01-31,0.03750,0.80,360,0.00,775,0.2334,0.0906,2718.4,0,0,TRAD,24729.58,135000.0,0.80,0,1,0,0,0,0,0,0,1,0,0,0,0,ServicingRetained,108000.0,NaN,-0.110,-0.120,0,0,0,0,0,0,0,0.00,0.0000,Not Prepaid,0,0.6,0.6,3.13,3.86,0.0,4.0,129408.424228,780.575772,0.000000,2015-01-02,None,Cash Out - Refinance,0,0.570000,2015-01,0.000000,1.000000,0,0,0,1,0,0,1,0,0,1
1,1568633,2015-02-28,0.03750,0.80,360,0.00,775,0.2334,0.0906,2718.4,0,0,TRAD,24729.58,135000.0,0.80,0,1,0,0,0,0,0,1,2,0,0,0,0,ServicingRetained,108000.0,108000.00,0.080,-0.120,1,0,0,0,0,0,0,0.00,0.0000,Not Prepaid,0,0.5,0.6,2.99,3.67,0.0,3.9,130189.000000,780.575772,780.575772,2015-01-02,None,Cash Out - Refinance,0,0.595719,2015-02,0.000000,1.000000,0,0,0,1,0,0,1,0,0,1
2,1568633,2015-03-31,0.03750,0.80,360,0.00,775,0.2334,0.0906,2718.4,0,0,TRAD,24729.58,135000.0,0.80,0,1,0,0,0,0,0,2,3,0,0,0,0,ServicingRetained,108000.0,108000.00,0.040,-0.120,2,0,0,0,0,0,0,0.00,0.0000,Not Prepaid,0,0.6,0.6,3.01,3.71,0.0,3.9,132165.000000,1976.000000,2756.575772,2015-01-02,None,Cash Out - Refinance,0,0.707263,2015-03,0.000000,1.000000,0,0,0,1,0,0,1,0,0,1
3,1568633,2015-04-30,0.03750,0.80,360,0.00,775,0.2334,0.0906,2718.4,0,0,TRAD,24729.58,135000.0,0.80,0,1,0,0,0,0,0,3,4,0,0,0,0,ServicingRetained,108000.0,107837.33,-0.020,-0.120,2,0,0,0,0,0,0,41.25,0.0004,Curtailment,1,0.6,0.6,3.04,3.77,0.0,3.9,134582.000000,2417.000000,5173.575772,2015-01-02,None,Cash Out - Refinance,0,0.713536,2015-04,0.000341,0.998494,0,0,0,1,0,0,1,0,0,1
4,1568633,2015-05-31,0.03750,0.80,360,0.00,775,0.2334,0.0906,2718.4,0,0,TRAD,24729.58,135000.0,0.80,0,1,0,0,0,0,0,4,5,0,0,0,0,ServicingRetained,108000.0,107632.77,0.080,-0.120,3,0,0,0,0,0,0,41.24,0.0004,Curtailment,1,0.6,0.6,2.94,3.67,0.0,3.8,135608.000000,1026.000000,6199.575772,2015-01-02,None,Cash Out - Refinance,0,0.626286,2015-05,0.000393,0.996600,0,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6465492,2019869,2019-12-31,0.03750,0.53,360,0.00,809,0.1547,0.1409,90.0,0,0,TRAD,9082.00,230000.0,0.53,0,1,0,0,0,0,0,1,1,0,0,0,0,ServicingRetained,120000.0,NaN,0.050,0.090,1,0,0,0,0,0,0,0.00,0.0000,Not Prepaid,0,0.5,0.6,3.16,3.70,0.0,4.7,176741.000000,1039.000000,0.000000,2019-12-31,None,Cash Out - Refinance,0,0.676032,2019-12,0.012968,1.000000,0,0,1,0,0,0,1,0,0,0
6465494,2019870,2019-12-31,0.03625,0.77,360,0.00,702,0.4794,0.2503,1439.0,0,1,TRAD,6281.58,225000.0,0.77,0,1,0,0,0,0,0,0,1,0,0,0,0,ServicingRetained,172000.0,NaN,-0.075,-0.105,0,0,0,0,0,0,0,0.00,0.0000,Not Prepaid,0,0.5,0.6,3.16,3.70,0.0,3.8,258679.000000,-959.000000,171.000000,2019-12-31,None,Limited Cash Out - Refinance,0,0.676032,2019-12,0.012968,1.000000,0,0,0,1,0,0,1,0,0,0
6465496,2

In [2]:
fullclean['IND_TPO'] = pd.Categorical(fullclean['IND_TPO'])
fullclean['IND_OBS2'] = pd.Categorical(fullclean['IND_OBS2'])
fullclean['IND_TPO'] = fullclean['IND_TPO'].cat.codes
fullclean['IND_OBS2'] = fullclean['IND_OBS2'].cat.codes

In [3]:
pd.set_option('display.max_columns', 125)
pd.set_option('display.max_rows', 501)

nullsanddistinct = []

for i in range(len(fullclean.columns)):
    nullsanddistinct.append([fullclean.columns[i] , fullclean[fullclean.columns[i]].isna().sum(), fullclean[fullclean.columns[i]].nunique(), fullclean[fullclean.columns[i]].dtypes]  )

nullsanddistinct = pd.DataFrame(nullsanddistinct)
nullsanddistinct.columns = ['Variable', ' Number of Nulls', 'Unique Variables', 'Datatypes']
nullsanddistinct

KeyboardInterrupt: 

In [3]:


####################AAAAALLLLLTTTT############################

#dont need
fullclean = fullclean[fullclean['PrepayUPB'] >= 0]
fullclean = fullclean[fullclean['SMM'] >= 0]
fullclean.drop(['LoanNumber', 'PayDate','SMM_Flag','TotalOtherDebt', 'ServicingModelName',
                'PRODUCT','FundingDate','SubProgramCode','LoanPurposeDescr','PaydateABV','LoantoValue','IND_MonthofDefault'], axis = 1, inplace = True )

fullclean.dropna(how = 'any', inplace = True)

In [3]:
Inds = []
SevIm = []
for i in range(len(fullclean.columns)):
    if fullclean[fullclean.columns[i]].nunique() == 2:
        Inds.append([fullclean.columns[i], max(fullclean[fullclean.columns[i]].value_counts())/min(fullclean[fullclean.columns[i]].value_counts())])
Inds = pd.DataFrame(Inds)

Inds.columns = ['Name','Ratio']
h  =  Inds[Inds['Name'] == 'IND_FULL'].index
Inds.drop(h, inplace= True)

Inds

fullclean.drop(Inds.loc[Inds['Ratio'] > 50]['Name'], inplace= True, axis = 1) # remove highly imbalance indicators

In [7]:
list(Inds.loc[Inds['Ratio']>50, 'Name'])

['CurrentFlag',
 'DLQ30Flag',
 'DLQ60Flag',
 'DLQ90OrPlusFlag',
 'REOForeclosureFlag',
 'DLQDays',
 'IND_MonthofDefault',
 'IND_REOFCL_MOD',
 'IND_180D_MOD',
 'IND_LGD_MOD',
 'IND_HLC_MOD']

In [4]:
#dont need
fullclean = fullclean[fullclean['PrepayUPB'] >= 0]
fullclean = fullclean[fullclean['SMM'] >= 0]
fullclean.drop(['LoanNumber', 'PayDate','SMM_Flag','TotalOtherDebt', 'ServicingModelName',
                'PRODUCT','FundingDate','SubProgramCode','LoanPurposeDescr','PaydateABV','LoantoValue'], axis = 1, inplace = True )

#high cor
fullclean.drop(['BegMonthBal', 'RollingDLQ30Flag', 
                'RollingDLQ60Flag', 'FreddieMac_Pts_15Yr', 'RollingDLQ90OrPlusFlag', 
                'RollingREOForeclosureFlag','RollingDLQDays'], axis = 1, inplace = True )

fullclean.dropna(how = 'any', inplace = True)

In [14]:
fullclean.to_csv('ggg.csv')

Tensorflow

In [138]:
fullclean2 = fullclean

In [139]:
target = fullclean2.pop('IND_FULL')

In [7]:
greaterthantwo = []
for i in range(len(fullclean2.columns)):
    if fullclean[fullclean2.columns[i]].nunique() > 2:
        greaterthantwo.append(fullclean2.columns[i])
#fullclean[greaterthantwo] = sklearn.preprocessing.normalize(fullclean[greaterthantwo], axis = 0)
scaler = MinMaxScaler(feature_range=(0,1))
fullclean2[greaterthantwo] = scaler.fit_transform(fullclean2[greaterthantwo])

In [140]:
dataset = tf.data.Dataset.from_tensor_slices(( fullclean2.values, target ))

In [48]:
for feat, targ in dataset.take(5):
    print('f:{} , T:{}'.format(feat,targ))

f:[4.12500000e-02 8.00000000e-01 3.60000000e+02 0.00000000e+00
 8.05000000e+02 7.38000000e-02 7.37000000e-02 0.00000000e+00
 1.00000000e+00 4.17050000e+04 5.00000000e+05 0.00000000e+00
 1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 2.50000000e+01 1.20000000e+01
 4.00000000e+05 3.25000000e-01 2.15000000e-01 1.10000000e+01
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 5.00000000e-01 3.24000000e+00 3.80000000e+00 0.00000000e+00
 4.30000019e+00 2.09000000e+02 9.65600000e+03 4.27442024e-01
 8.29119598e-03 9.53114125e-01 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.00000000e+00 0.00000000e+00 0.00000000e+00
 1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00] , T:0
f:[ 3.37500000e-02  8.00000000e-01  3.60000000e+02  0.00000000e+00
  7.58000000e+02  3.50300000e-01  2.12100000e-01  0.00000000e+00
  0.00000000e+00  1.29962000e+04  3.70000000e+05  0.00000000e+00
 

In [141]:
dataset = dataset.shuffle(len(fullclean2))

In [142]:
train_dataset = dataset.take(1000).batch(1)
test_dataset = dataset.take(1000)                     

In [143]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(2, activation=tf.nn.softmax))

In [144]:
model.compile(optimizer='adam', #adam is default there are others
        loss='sparse_categorical_crossentropy',	#loss = degree of error - which nn is trying to minimze -this is one type of error
        metrics=['accuracy']) 

In [145]:
model.fit(train_dataset, epochs = 3)

AttributeError: 'google.protobuf.pyext._message.RepeatedCompositeCo' object has no attribute 'append'

In [10]:
fullclean[ fullclean['IND_FULL'] == 1]['Seasoning'].value_counts()

12    1679
11    1666
13    1628
8     1623
14    1622
7     1585
9     1576
10    1540
15    1526
16    1423
17    1263
6     1232
5     1198
18    1147
20    1058
19    1051
21     959
27     923
25     910
26     900
24     875
23     860
22     839
28     775
4      758
29     740
30     720
31     679
32     642
33     605
35     588
36     573
37     570
34     553
38     476
39     426
40     425
3      417
41     364
2      320
43     312
42     302
44     291
45     276
47     270
49     266
46     252
48     246
50     223
52     208
51     206
53     172
1      154
54     153
55     113
56      78
57      72
58      43
59      10
0        8
Name: Seasoning, dtype: int64

In [117]:
fullclean2

,InterestRate,LoantoValue,Original_Amortization_Term,PMIPercent,Original_FICO_Min,TotalDebtratio,IncomePercentage,IND_TPO,IND_OBS2,TOTALMonthlyIncome,...,XTRAIND,EastIND,SouthIND,MidwestIND,WestIND,PurchaseIND,CashoutRefIND,RateRefIND,NonSingleFamilyIND,NonprimaryIND
0,0.000385,0.000422,0.000444,0.000000,0.000414,0.000292,0.000170,0,0,0.000242,...,0,0,0,1,0,0,1,0,0,1
1,0.000385,0.000422,0.000444,0.000000,0.000414,0.000292,0.000170,0,0,0.000242,...,0,0,0,1,0,0,1,0,0,1
2,0.000385,0.000422,0.000444,0.000000,0.000414,0.000292,0.000170,0,0,0.000242,...,0,0,0,1,0,0,1,0,0,1
3,0.000385,0.000422,0.000444,0.000000,0.000414,0.000292,0.000170,0,0,0.000242,...,0,0,0,1,0,0,1,0,0,1
4,0.000385,0.000422,0.000444,0.000000,0.000414,0.000292,0.000170,0,0,0.000242,...,0,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6465492,0.000385,0.000280,0.000444,0.000000,0.000432,0.000194,0.000264,0,0,0.000089,...,0,0,1,0,0,0,1,0,0,0
6465494,0.000372,0.000406,0.000444,0.000000,0.000375,0.000600,0.000470,0,1,0.000062,...,0,0,0,1,0,0,1,0,0,0
6465496,0.000475,0.000422,0.000444,0.000000,0.000380,0.000440,0.000377,0,0,0.000120,...,0,0,1,0,0,0,1,0,1,0
6465498,0.000398,0.000491,0.000444,0.000971,0.000358,0.000378,0.000200,0,0,0.000136,...,0,0,0,1,0,1,0,0,0,0


Sklearn

In [4]:
fullclean2 = fullclean.copy()

In [42]:
##FROM HERE

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix
import random

In [44]:
greaterthantwo = []
twoo = []
for i in range(len(fullclean2.columns)):
    if fullclean2[fullclean2.columns[i]].nunique() > 2:
        greaterthantwo.append(fullclean2.columns[i])
    else:
        twoo.append(fullclean2.columns[i])
#fullclean[greaterthantwo] = sklearn.preprocessing.normalize(fullclean[greaterthantwo], axis = 0)
scaler = MinMaxScaler(feature_range=(0,1))

fullclean2[greaterthantwo] = scaler.fit_transform(fullclean2[greaterthantwo])

#fullclean2[greaterthantwo] = sklearn.preprocessing.normalize(fullclean2[greaterthantwo], axis = 0)

MemoryError: Unable to allocate array with shape (6215835,) and data type float64

In [21]:
fullclean22 = fullclean2[2000000:6000000].copy()

In [22]:
target = fullclean22.pop('IND_FULL')
X_train, X_test, y_train, y_test = train_test_split(fullclean22, target)

In [15]:
del fullclean2

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix
import random

In [6]:
fullclean2 = fullclean[1000000:6000000].copy()

In [7]:
greaterthantwo = []
twoo = []
for i in range(len(fullclean2.columns)):
    if fullclean2[fullclean2.columns[i]].nunique() > 2:
        greaterthantwo.append(fullclean2.columns[i])
    else:
        twoo.append(fullclean2.columns[i])
#fullclean[greaterthantwo] = sklearn.preprocessing.normalize(fullclean[greaterthantwo], axis = 0)

scaler = MinMaxScaler(feature_range=(0,1))
fullclean2[greaterthantwo] = scaler.fit_transform(fullclean2[greaterthantwo])

#fullclean2[greaterthantwo] = sklearn.preprocessing.normalize(fullclean2[greaterthantwo], axis = 0)

In [8]:
target = fullclean2.pop('IND_FULL')
X_train, X_test, y_train, y_test = train_test_split(fullclean2, target, shuffle = False, train_size = .75)

In [22]:
print(y_train.value_counts()[1] , y_train.value_counts()[0])
print(y_train.value_counts()[1]/y_train.value_counts()[0])

20819 3729181
0.005582727145719127


In [9]:
percentage = .3
a = int(    y_train.value_counts()[0] -   (y_train.value_counts()[1]/percentage)            )
#a = 1417973
by = random.sample(list(y_train.loc[y_train == 0].index ),a)

In [10]:
y_train.drop(by, inplace = True)
X_train.drop(by, inplace = True)

c:\program files\python36\lib\site-packages\pandas\core\frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [11]:
hl1 = 64
hl2 = 32

hl3 = 16

it = 10000
minb = 200


mlp = MLPClassifier(hidden_layer_sizes=(hl1,hl2),max_iter=it,verbose=False, batch_size = minb)

In [12]:
mlp.fit(X_train,y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size=200, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(64, 32), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=10000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [13]:
#test

predictions = mlp.predict(X_test)
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(sklearn.metrics.roc_auc_score(y_test,predictions))
#precision is ppv takes support into account
#recall is sensitiviy (total predicted/total actual)
#columns are predicted - rows are truth

[[881184 354070]
 [  5189   9557]]
              precision    recall  f1-score   support

           0       0.99      0.71      0.83   1235254
           1       0.03      0.65      0.05     14746

    accuracy                           0.71   1250000
   macro avg       0.51      0.68      0.44   1250000
weighted avg       0.98      0.71      0.82   1250000

0.680735278676025


In [14]:
#TRAINS

predictions = mlp.predict(X_train)
print(confusion_matrix(y_train,predictions))
print(classification_report(y_train,predictions))
print(sklearn.metrics.roc_auc_score(y_train,predictions))

[[36865  4705]
 [11475  9310]]
              precision    recall  f1-score   support

           0       0.76      0.89      0.82     41570
           1       0.66      0.45      0.54     20785

    accuracy                           0.74     62355
   macro avg       0.71      0.67      0.68     62355
weighted avg       0.73      0.74      0.73     62355

0.667368294443108


In [131]:
dep = pd.DataFrame(((predictions) != y_test) & (predictions == 0)).loc[
    pd.DataFrame(((predictions) != y_test) & (predictions == 0))['IND_FULL'] == True ].index

In [47]:
reset %

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
Don't know how to reset  %, please run `%reset?` for details
